# Initial data

In [151]:
import collections

collections.Iterable = collections.abc.Iterable

In [152]:
import pandas as pd
import dill

In [153]:
pd.set_option('display.max_columns', None)

In [154]:
with open('./data/data.pkl', 'rb') as f:
    childcare_inspection_data = dill.load(f)

In [155]:


childcare_inspection_data.drop_duplicates(inplace=True)

childcare_inspection_data.drop(['Legal Name', 'Building', 'Street', 'ZipCode',
                               'Permit Number', 'Building Identification Number', 'Actual'], axis=1, inplace=True)

childcare_inspection_data['Center Name'] = childcare_inspection_data['Center Name'].replace(
    '\x1a', "'", regex=True)

# childcare_inspection_data.dropna(subset=['Inspection Date'],inplace=True )
childcare_inspection_data[[ 'Inspection Date']] = childcare_inspection_data[[
     'Inspection Date']].apply(pd.to_datetime, errors='coerce')

childcare_inspection_data['Date Permitted'] = childcare_inspection_data['Date Permitted'].dt.strftime('%Y-%m-%d')
childcare_inspection_data['Permit Expiration'] = childcare_inspection_data['Permit Expiration'].dt.strftime('%Y-%m-%d')



childcare_inspection_data.sort_values(
    ['Day Care ID', 'Inspection Date'], ascending=[True, False], inplace=True)

In [156]:
childcare_inspection_data.to_csv('./data/childcare_inspection_data.csv')

In [157]:
import dill

with open('./data/childcare_inspection_data.pkl', 'wb') as g:
    dill.dump(childcare_inspection_data, g)

In [158]:
with open('./data/childcare_inspection_data.pkl', 'rb') as g:
    childcare_inspection_data = dill.load(g)

In [159]:
childcare_inspection_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23307 entries, 13059 to 15229
Data columns (total 27 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   Center Name                                  23307 non-null  object        
 1   Borough                                      23307 non-null  object        
 2   Phone                                        23299 non-null  object        
 3   Permit Expiration                            23307 non-null  object        
 4   Status                                       23307 non-null  object        
 5   Age Range                                    21014 non-null  object        
 6   Maximum Capacity                             23307 non-null  int64         
 7   Day Care ID                                  23307 non-null  object        
 8   Program Type                                 23307 non-null  object     

# Inspection results

In [160]:
inspection_data = childcare_inspection_data[['Day Care ID', 'Inspection Date', 'Violation Category',
                                             'Health Code Sub Section', 'Violation Status', 'Regulation Summary']]

inspection_data.drop_duplicates(inplace=True)

C:\Users\abidg\AppData\Local\Temp\ipykernel_33072\2799740336.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inspection_data.drop_duplicates(inplace=True)


In [161]:
inspection_data_0 = inspection_data.copy()

inspection_data_0['Inspection Date'] = inspection_data_0['Inspection Date'].dt.strftime('%Y-%m-%d')
inspection_data_0[['Inspection Date']] = inspection_data_0[[
    'Inspection Date']].fillna('No inspection data Avialable')

inspection_data_0[['Regulation Summary', 'Violation Category',
                   'Health Code Sub Section', 'Violation Status']] = inspection_data_0[['Regulation Summary', 'Violation Category',
                                                                                        'Health Code Sub Section', 'Violation Status']].fillna('N/A')

inspection_data_0.reset_index(inplace=True)
inspection_data_viol_only = inspection_data_0.set_index(
    ['index', 'Day Care ID']).apply(dict, axis=1).reset_index(name='inspection_data_per_violation')

inspection_data_1 = inspection_data_0.merge(
    inspection_data_viol_only, how='left')

In [162]:
inspection_results = inspection_data_1.groupby(['Day Care ID'])['inspection_data_per_violation'].apply(
    list).reset_index(name='inspection_data_per_inspection')

inspection_results.head()

,Day Care ID,inspection_data_per_inspection
0,DC1000,"[{'Inspection Date': '2022-03-08', 'Violation ..."
1,DC1021,"[{'Inspection Date': '2022-07-19', 'Violation ..."
2,DC10244,"[{'Inspection Date': '2022-06-16', 'Violation ..."
3,DC1025,"[{'Inspection Date': '2022-02-09', 'Violation ..."
4,DC1030,"[{'Inspection Date': '2022-06-16', 'Violation ..."


In [163]:
import dill

with open('./data/inspection_results.pkl', 'wb') as k:
    dill.dump(inspection_results, k)


In [164]:
    
with open('./data/inspection_results.pkl', 'rb') as k:
    inspection_results = dill.load(k)

# Inspection calculations

In [165]:
# rows where inspection date available
inspection_data_valid = inspection_data[inspection_data['Inspection Date'].notnull(
)]

# rows where inspection date not available
inspection_data_not_valid = inspection_data[inspection_data['Inspection Date'].isnull(
)]

In [166]:
inspection_calculations = inspection_data_valid.groupby(['Day Care ID']).agg(
    inspection_day_count=('Inspection Date', 'nunique')).reset_index()

inspection_days_with_violation = inspection_data_valid[inspection_data['Violation Category'].notnull()].groupby(
    ['Day Care ID'], as_index=False).agg(inspection_days_with_violation=('Inspection Date', 'nunique'))

inspection_calculations = inspection_calculations.merge(
    inspection_days_with_violation, how='left', on='Day Care ID').fillna(0)

C:\Users\abidg\AppData\Local\Temp\ipykernel_33072\2574236262.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  inspection_days_with_violation = inspection_data_valid[inspection_data['Violation Category'].notnull()].groupby(


In [167]:
violation_count = inspection_data_valid.groupby(['Day Care ID']).agg(
    violation_count=('Violation Category', 'value_counts')).reset_index()
violation_count_final = pd.pivot(violation_count, index='Day Care ID',
                                 columns='Violation Category', values='violation_count').reset_index().fillna(0)

violation_count_final['TOTAL VIOLATION COUNT'] = violation_count_final.iloc[:, -
                                                                            3:].sum(axis=1)
inspection_calculations = inspection_calculations.merge(
    violation_count_final, how='left', on='Day Care ID').fillna(0)

In [168]:
inspection_calculations

,Day Care ID,inspection_day_count,inspection_days_with_violation,CRITICAL,GENERAL,PUBLIC HEALTH HAZARD,TOTAL VIOLATION COUNT
0,DC1000,8,3.0,5.0,2.0,0.0,7.0
1,DC1021,19,12.0,11.0,10.0,1.0,22.0
2,DC10244,8,2.0,2.0,0.0,0.0,2.0
3,DC1025,5,2.0,2.0,2.0,0.0,4.0
4,DC1030,11,6.0,2.0,10.0,0.0,12.0
...,...,...,...,...,...,...,...
2934,DC955,6,2.0,3.0,1.0,2.0,6.0
2935,DC956,5,2.0,1.0,1.0,0.0,2.0
2936,DC959,8,3.0,1.0,2.0,0.0,3.0
2937,DC962,6,1.0,1.0,1.0,0.0,2.0


In [169]:
def flatten_multi_index(df):
    mi = df.columns
#     suffixes, prefixes = mi.levels
    col_names = [' '.join(i) if len(i[1]) > 0 else i[0] for i in mi]
    df.columns = col_names
    return df


open_violation_count = flatten_multi_index(pd.pivot_table(inspection_data, index='Day Care ID', columns=[
                                           'Violation Category', 'Violation Status'], aggfunc='size', fill_value=0).reset_index())

open_violation_count.drop([i for i in open_violation_count.columns if (
    'CORRECTED' in i)], axis=1, inplace=True)

open_violation_count['TOTAL OPEN VIOLATIONS'] = open_violation_count.iloc[:, -
                                                                          3:].sum(axis=1)
inspection_calculations = inspection_calculations.merge(
    open_violation_count, how='left', on='Day Care ID').fillna(0)

In [170]:
inspection_calculations['violation_day_rate'] = inspection_calculations['inspection_days_with_violation'] / \
    inspection_calculations['inspection_day_count']
inspection_calculations['critical_per_insp'] = inspection_calculations['CRITICAL'] / \
    inspection_calculations['inspection_day_count']
inspection_calculations['general_per_insp'] = inspection_calculations['GENERAL'] / \
    inspection_calculations['inspection_day_count']
inspection_calculations['public_health_hazard_per_insp'] = inspection_calculations['PUBLIC HEALTH HAZARD'] / \
    inspection_calculations['inspection_day_count']
inspection_calculations['critical_open_per_insp'] = inspection_calculations['CRITICAL OPEN'] / \
    inspection_calculations['inspection_day_count']
inspection_calculations['general_open_per_insp'] = inspection_calculations['GENERAL OPEN'] / \
    inspection_calculations['inspection_day_count']
inspection_calculations['public_health_hazard_open_per_insp'] = inspection_calculations['PUBLIC HEALTH HAZARD OPEN'] / \
    inspection_calculations['inspection_day_count']

In [171]:
inspection_calculations['critical_prob'] = inspection_calculations['CRITICAL'] / \
    inspection_calculations['CRITICAL'].sum()
inspection_calculations['general_prob'] = inspection_calculations['GENERAL'] / \
    inspection_calculations['GENERAL'].sum()
inspection_calculations['public_health_hazard_prob'] = inspection_calculations['PUBLIC HEALTH HAZARD'] / \
    inspection_calculations['PUBLIC HEALTH HAZARD'].sum()
inspection_calculations['critical_open_prob'] = inspection_calculations['CRITICAL OPEN'] / \
    inspection_calculations['CRITICAL OPEN'].sum()
inspection_calculations['general_open_prob'] = inspection_calculations['GENERAL OPEN'] / \
    inspection_calculations['GENERAL OPEN'].sum()
inspection_calculations['public_health_hazard_open_prob'] = inspection_calculations['PUBLIC HEALTH HAZARD OPEN'] / \
    inspection_calculations['PUBLIC HEALTH HAZARD OPEN'].sum()

In [172]:
inspection_calculations['critical_score'] = inspection_calculations['critical_per_insp'] * \
    inspection_calculations['critical_prob'] * \
    inspection_calculations['violation_day_rate']
inspection_calculations['general_score'] = inspection_calculations['general_per_insp'] * \
    inspection_calculations['general_prob'] * \
    inspection_calculations['violation_day_rate']
inspection_calculations['public_health_hazard_score'] = inspection_calculations['public_health_hazard_per_insp'] * \
    inspection_calculations['public_health_hazard_prob'] * \
    inspection_calculations['violation_day_rate']
inspection_calculations['critical_open_score'] = inspection_calculations['critical_open_per_insp'] * \
    inspection_calculations['critical_open_prob'] * \
    inspection_calculations['violation_day_rate']
inspection_calculations['general_open_score'] = inspection_calculations['general_open_per_insp'] * \
    inspection_calculations['general_open_prob'] * \
    inspection_calculations['violation_day_rate']
inspection_calculations['public_health_hazard_open_score'] = inspection_calculations['public_health_hazard_open_per_insp'] * \
    inspection_calculations['public_health_hazard_open_prob'] * \
    inspection_calculations['violation_day_rate']

In [173]:
inspection_calculations['violation_score'] = inspection_calculations['public_health_hazard_score'] * 35 \
    + inspection_calculations['public_health_hazard_open_score'] * 35  \
    + inspection_calculations['critical_score'] * 10 \
    + inspection_calculations['critical_open_score'] * 10 \
    + inspection_calculations['general_score'] * 5 \
    + inspection_calculations['general_open_score'] * 5

In [174]:

inspection_score = inspection_calculations[['Day Care ID', 'inspection_day_count', 'inspection_days_with_violation',
       'CRITICAL', 'GENERAL', 'PUBLIC HEALTH HAZARD', 'TOTAL VIOLATION COUNT',
       'violation_score']]

In [175]:
inspection_score_no_inspection = inspection_score[:len(inspection_data_not_valid)].copy()
inspection_score_no_inspection.iloc[:,:] = 0

inspection_score_no_inspection['Day Care ID'] = inspection_data_not_valid.reset_index()['Day Care ID']

inspection_score_final = pd.concat([inspection_score, inspection_score_no_inspection], axis =0)
inspection_score_final['Safety_score'] = 10 - inspection_score_final['violation_score']
inspection_score_final.sort_values(['Safety_score','inspection_day_count'], ascending=[False, False] )

,Day Care ID,inspection_day_count,inspection_days_with_violation,CRITICAL,GENERAL,PUBLIC HEALTH HAZARD,TOTAL VIOLATION COUNT,violation_score,Safety_score
609,DC184,10,0.0,0.0,0.0,0.0,0.0,0.000000,10.000000
1810,DC32600,10,0.0,0.0,0.0,0.0,0.0,0.000000,10.000000
1946,DC33765,10,0.0,0.0,0.0,0.0,0.0,0.000000,10.000000
2080,DC34868,10,0.0,0.0,0.0,0.0,0.0,0.000000,10.000000
229,DC1373,9,0.0,0.0,0.0,0.0,0.0,0.000000,10.000000
...,...,...,...,...,...,...,...,...,...
213,DC1308,7,5.0,9.0,3.0,8.0,20.0,1.213807,8.786193
2640,DC39359,2,2.0,3.0,0.0,5.0,8.0,1.636186,8.363814
2610,DC39118,1,1.0,2.0,2.0,3.0,7.0,1.819176,8.180824
2847,DC41121,1,1.0,3.0,0.0,3.0,6.0,1.958073,8.041927


In [176]:
import dill

with open('./data/inspection_score_final.pkl', 'wb') as p:
    dill.dump(inspection_score_final, p)
    


In [177]:
    
with open('./data/inspection_score_final.pkl', 'rb') as p:
    inspection_score_final = dill.load(p)

# Coordinate data

In [178]:
with open('./data/data.pkl', 'rb') as f:
    initial_data = dill.load(f)

initial_data.drop_duplicates(inplace=True)


initial_data['Address'] = initial_data['Building'].fillna('') + ' ' + \
    initial_data['Street'] + ', ' + \
    initial_data['Borough']

childcare_addresses = initial_data[[
    'Day Care ID', 'Center Name', 'Building', 'Street', 'Borough', 'ZipCode', 'Address']].drop_duplicates().reset_index(drop=True)


missing_building_index = list(
    childcare_addresses[childcare_addresses['Building'].isnull()].index)

childcare_addresses.loc[missing_building_index,
                        'Address'] = childcare_addresses['Center Name'] + ', ' + childcare_addresses['Address']

In [179]:
import os
data_path = 'data'
if not os.path.exists(data_path):
    os.mkdir(data_path)

childcare_addresses.to_csv('data/childcare_addresses.csv')

# {name: len(childcare_inspection_data[name].unique())
#  for name in childcare_inspection_data.columns}

In [180]:
### Use google_batch_geocode.py to  generate childcare_addresses_lat_long.csv file 

In [181]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

centers = pd.read_csv('./data/childcare_addresses.csv', index_col=0)

centers_lat_long_from_google = pd.read_csv(
    './data/childcare_addresses_lat_long.csv', index_col=0)

centers_lat_long_merged = centers.merge(
    centers_lat_long_from_google, how='inner', on='Day Care ID')


centers_lat_long = centers_lat_long_merged[['Day Care ID', 'Center Name', 'formatted_address','Borough', 'latitude', 'longitude', 
                                          'google_place_id','postcode']]

centers_lat_long['coordinates'] = list(zip(centers_lat_long.latitude, centers_lat_long.longitude))

centers_lat_long['_coordinate_Point'] = [Point(xy) for xy in zip(centers_lat_long.latitude, centers_lat_long.longitude)]



C:\Users\abidg\AppData\Local\Temp\ipykernel_33072\2658724326.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  centers_lat_long['coordinates'] = list(zip(centers_lat_long.latitude, centers_lat_long.longitude))
C:\Users\abidg\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\abidg\AppData\Local\Temp\ipykernel_33072\2658724326.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [182]:
centers_lat_long

,Day Care ID,Center Name,formatted_address,Borough,latitude,longitude,google_place_id,postcode,coordinates,_coordinate_Point
0,DC10485,DIVINE MERCY CATHOLIC ACADEMY,"101-60 92nd St, Queens, NY 11416, USA",QUEENS,40.682292,-73.848242,ChIJMSqHTvFdwokRe6jPTDBV4GI,11416,"(40.6822919, -73.8482418)",POINT (40.6822919 -73.8482418)
1,DC1643,EAST TREMONT CHILD CARE AND DEVELOPMENT CENTER,"1811 Crotona Ave, The Bronx, NY 10457, USA",BRONX,40.843201,-73.894351,ChIJkdMA0WD0wokR7mfM2Z8ieLI,10457,"(40.8432011, -73.8943511)",POINT (40.8432011 -73.8943511)
2,DC2652,THE JACK AND JILL SCHOOL AT ST. GEORGE'S CHURCH,"209 E 16th St, New York, NY 10003, USA",MANHATTAN,40.734350,-73.984961,ChIJ_f9zvZ9ZwokRVbgl-eLi4pg,10003,"(40.7343503, -73.9849615)",POINT (40.7343503 -73.9849615)
3,DC24039,JCC BROOKLYN WINDSOR TERRACE,"1224 Prospect Ave, Brooklyn, NY 11218, USA",BROOKLYN,40.653248,-73.976559,ChIJp5V8LiJbwokRQFhb3ANx3pk,11218,"(40.6532485, -73.9765589)",POINT (40.6532485 -73.9765589)
4,DC17217,TREY WHITFIELD SCHOOL,"17 Hinsdale St, Brooklyn, NY 11207, USA",BROOKLYN,40.674922,-73.900985,ChIJmceaWUVcwokRZrr1ftAR9ts,11207,"(40.6749222, -73.9009846)",POINT (40.6749222 -73.9009846)
...,...,...,...,...,...,...,...,...,...,...
3078,DC39473,ZETA CHARTER SCHOOLS,"652 W 187th St, New York, NY 10033, USA",MANHATTAN,40.853696,-73.932865,ChIJq6pqUhz0wokRESiikZ5KrzA,10033,"(40.8536963, -73.9328649)",POINT (40.8536963 -73.9328649)
3079,DC40430,Chabad Lubavitch of Staten Island,"275 Martling Ave, Staten Island, NY 10314, USA",STATEN ISLAND,40.621523,-74.123537,ChIJMbILU39OwokRGtMxnybl8Sg,10314,"(40.6215227, -74.1235372)",POINT (40.6215227 -74.1235372)
3080,DC1425,ST. JOSEPH PRESCHOOL,"420 E 87th St, New York, NY 10128, USA",MANHATTAN,40.777066,-73.947962,ChIJa6axfLpYwokRC0oQyHL13h4,10128,"(40.7770657, -73.9479624)",POINT (40.7770657 -73.9479624)
3081,DC19550,VILLA MARIA ACADEMY,"3335 Country Club Rd, The Bronx, NY 10465, USA",BRONX,40.841443,-73.816542,ChIJc4cVvWeLwokRdzUGOawRfls,10465,"(40.8414434, -73.8165423)",POINT (40.8414434 -73.8165423)


In [183]:
import dill

with open('./data/centers_lat_long.pkl', 'wb') as a:
    dill.dump(centers_lat_long, a)
    


In [184]:
    
with open('./data/centers_lat_long.pkl', 'rb') as a:
    centers_lat_long = dill.load(a)

# Add category Column to data

In [185]:
childcare_inspection_data.columns

Index(['Center Name', 'Borough', 'Phone', 'Permit Expiration', 'Status',
       'Age Range', 'Maximum Capacity', 'Day Care ID', 'Program Type',
       'Facility Type', 'Child Care Type', 'URL', 'Date Permitted',
       'Violation Rate Percent', 'Average Violation Rate Percent',
       'Total Educational Workers', 'Average Total Educational Workers',
       'Public Health Hazard Violation Rate',
       'Average Public Health Hazard Violation Rate',
       'Critical Violation Rate', 'Average Critical Violation Rate',
       'Inspection Date', 'Regulation Summary', 'Violation Category',
       'Health Code Sub Section', 'Violation Status',
       'Inspection Summary Result'],
      dtype='object')

In [186]:
childcare_information_data = childcare_inspection_data.groupby(['Day Care ID']).first().drop(['Inspection Date', 'Regulation Summary', 'Violation Category',
       'Health Code Sub Section', 'Violation Status',
       'Inspection Summary Result'],axis=1).reset_index()

In [187]:
childcare_information_data['Max Capacity Range'] = pd.cut(childcare_information_data['Maximum Capacity'], [-1, 10, 30,50,100,1300], labels=['<=10', '11-30', '30-50', '50-100', '>100'])
childcare_information_data['Total Educational Workers Range'] = pd.cut(childcare_information_data['Total Educational Workers'], [-1, 1, 5,8,15,130], labels=['<=1', '1-5', '5-8', '8-15', '>15'])

In [188]:
childcare_information_data['Total Educational Workers Range'] = pd.cut(childcare_information_data['Total Educational Workers'], [-1, 1, 5,8,15,130], labels=['<=1', '1-5', '5-8', '8-15', '>15'])
childcare_information_data

,Day Care ID,Center Name,Borough,Phone,Permit Expiration,Status,Age Range,Maximum Capacity,Program Type,Facility Type,Child Care Type,URL,Date Permitted,Violation Rate Percent,Average Violation Rate Percent,Total Educational Workers,Average Total Educational Workers,Public Health Hazard Violation Rate,Average Public Health Hazard Violation Rate,Critical Violation Rate,Average Critical Violation Rate,Max Capacity Range,Total Educational Workers Range
0,DC1000,BILLY MARTIN CHILD DEVELOPMENT CENTER,BROOKLYN,718-857-5630,2024-04-06,Permitted,None,70,PRESCHOOL,GDC,Child Care - Pre School,None,2004-03-30,50.0000,20.7182,7,11.1888,0.0,6.2024,50.0000,17.8904,50-100,5-8
1,DC1021,NUESTROS NINOS DAY CAR 1,BROOKLYN,718-963-1555,2023-08-07,Permitted,2 YEARS - 5 YEARS,162,PRESCHOOL,GDC,Child Care - Pre School,None,2003-11-24,0.0000,20.7182,33,11.1888,0.0,6.2024,0.0000,17.8904,>100,>15
2,DC10244,HERBERT G. BIRCH SERVICES,QUEENS,718-786-1104,2023-12-04,Permitted,2 YEARS - 5 YEARS,144,PRESCHOOL,GDC,Child Care - Pre School,None,2003-12-01,40.0000,20.7182,26,11.1888,0.0,6.2024,40.0000,17.8904,>100,>15
3,DC1025,"CNC CARE CENTER, INC.",BROOKLYN,718-284-0172,2023-09-14,Permitted,2 YEARS - 5 YEARS,19,PRESCHOOL,GDC,Child Care - Pre School,None,2005-09-14,20.0000,20.7182,3,11.1888,0.0,6.2024,20.0000,17.8904,11-30,1-5
4,DC1030,"PHIPPS NEIGHBORHOODS, INC.",BRONX,718-364-2496,2023-04-05,Permitted,2 YEARS - 5 YEARS,54,PRESCHOOL,GDC,Child Care - Pre School,phippsny.org,2002-11-25,0.0000,20.7182,11,11.1888,0.0,6.2024,0.0000,17.8904,50-100,8-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,DC955,"TUTOR TIME LEARNING CENTERS, LLC",BROOKLYN,718-680-6566,2023-08-10,Permitted,0 YEARS - 2 YEARS,36,INFANT TODDLER,GDC,Child Care - Infants/Toddlers,www.tutortime.com,2003-06-20,50.0000,17.6510,20,7.8268,25.0,5.5585,50.0000,15.2438,30-50,>15
3079,DC956,BAY RIDGE AVE. CHILD CARE CENTER (I/T),BROOKLYN,718-680-3544,2023-03-02,Permitted,0 YEARS - 2 YEARS,7,INFANT TODDLER,GDC,Child Care - Infants/Toddlers,None,2005-03-09,25.0000,17.6510,4,7.8268,0.0,5.5585,25.0000,15.2438,<=10,1-5
3080,DC959,THE CHILDREN'S CENTER @ SUNY BROOKLYN - P/S,BROOKLYN,718-221-6160,2023-07-05,Permitted,2 YEARS - 5 YEARS,42,PRESCHOOL,GDC,Child Care - Pre School,None,2003-05-08,16.6667,20.7182,6,11.1888,0.0,6.2024,16.6667,17.8904,30-50,5-8
3081,DC962,ACTION NURSERY,BROOKLYN,718-854-7777,2023-03-11,Permitted,2 YEARS - 5 YEARS,35,PRESCHOOL,GDC,Child Care - Pre School,None,2005-03-11,20.0000,20.7182,9,11.1888,0.0,6.2024,20.0000,17.8904,30-50,8-15


In [189]:
import dill

with open('./data/childcare_information_data.pkl', 'wb') as b:
    dill.dump(childcare_information_data, b)
    


In [190]:
    
with open('./data/childcare_information_data.pkl', 'rb') as b:
    childcare_information_data = dill.load(b)

# Join All Data

In [191]:
inspection_results.head(1)

,Day Care ID,inspection_data_per_inspection
0,DC1000,"[{'Inspection Date': '2022-03-08', 'Violation ..."


In [192]:
inspection_score_final.head(1)

,Day Care ID,inspection_day_count,inspection_days_with_violation,CRITICAL,GENERAL,PUBLIC HEALTH HAZARD,TOTAL VIOLATION COUNT,violation_score,Safety_score
0,DC1000,8,3.0,5.0,2.0,0.0,7.0,0.002925,9.997075


In [193]:
centers_lat_long.head(1)
centers_lat_long.columns

Index(['Day Care ID', 'Center Name', 'formatted_address', 'Borough',
       'latitude', 'longitude', 'google_place_id', 'postcode', 'coordinates',
       '_coordinate_Point'],
      dtype='object')

In [194]:
childcare_information_data.head(1)
childcare_information_data.columns

Index(['Day Care ID', 'Center Name', 'Borough', 'Phone', 'Permit Expiration',
       'Status', 'Age Range', 'Maximum Capacity', 'Program Type',
       'Facility Type', 'Child Care Type', 'URL', 'Date Permitted',
       'Violation Rate Percent', 'Average Violation Rate Percent',
       'Total Educational Workers', 'Average Total Educational Workers',
       'Public Health Hazard Violation Rate',
       'Average Public Health Hazard Violation Rate',
       'Critical Violation Rate', 'Average Critical Violation Rate',
       'Max Capacity Range', 'Total Educational Workers Range'],
      dtype='object')

In [195]:
m1 = childcare_information_data.merge(centers_lat_long[['Day Care ID', 'formatted_address',
                                                        'latitude', 'longitude', 'postcode', 'coordinates',
                                                        '_coordinate_Point']], on='Day Care ID')
m1.columns

Index(['Day Care ID', 'Center Name', 'Borough', 'Phone', 'Permit Expiration',
       'Status', 'Age Range', 'Maximum Capacity', 'Program Type',
       'Facility Type', 'Child Care Type', 'URL', 'Date Permitted',
       'Violation Rate Percent', 'Average Violation Rate Percent',
       'Total Educational Workers', 'Average Total Educational Workers',
       'Public Health Hazard Violation Rate',
       'Average Public Health Hazard Violation Rate',
       'Critical Violation Rate', 'Average Critical Violation Rate',
       'Max Capacity Range', 'Total Educational Workers Range',
       'formatted_address', 'latitude', 'longitude', 'postcode', 'coordinates',
       '_coordinate_Point'],
      dtype='object')

In [196]:
m2 = m1.merge(inspection_score_final, on= 'Day Care ID')

In [197]:
m3 = m2.merge(inspection_results, on= 'Day Care ID')

In [198]:
m3['ID_Name'] = m3['Day Care ID'] + \
    ': '+m3['Center Name']

In [199]:
final_data_for_app = m3[['Day Care ID', 'Center Name', 'ID_Name', 'formatted_address', 'Phone', 'URL', 'Borough', 'postcode',
                         'latitude', 'longitude',  'coordinates', '_coordinate_Point',
                         'Date Permitted', 'Status', 'Permit Expiration',
                         'Program Type', 'Facility Type', 'Child Care Type',
                         'Age Range', 'Maximum Capacity', 'Max Capacity Range',
                         'Total Educational Workers',  'Average Total Educational Workers', 'Total Educational Workers Range',
                         'Violation Rate Percent', 'Average Violation Rate Percent',
                         'Public Health Hazard Violation Rate', 'Average Public Health Hazard Violation Rate',
                         'Critical Violation Rate', 'Average Critical Violation Rate',
                         'inspection_day_count', 'inspection_days_with_violation',
                         'PUBLIC HEALTH HAZARD', 'CRITICAL', 'GENERAL',  'TOTAL VIOLATION COUNT',
                         'Safety_score', 'inspection_data_per_inspection']]

In [200]:
final_data_for_app.columns = ['Day Care ID', 'Center Name', 'ID_Name','Address',
                              'Phone', 'URL', 'Borough', 'Zipcode',
                              'Latitude', 'Longitude',  'Coordinates', '_coordinate_Point',
                              'Date Permitted', 'Status', 'Permit Expiration',
                              'Program Type', 'Facility Type', 'Child Care Type',
                              'Age Range', 'Maximum Capacity', 'Max Capacity Range',
                              'Total Educational Workers',  'Average Total Educational Workers', 'Total Educational Workers Range',
                              'Violation Rate Percent', 'Average Violation Rate Percent',
                              'Public Health Hazard Violation Rate', 'Average Public Health Hazard Violation Rate',
                              'Critical Violation Rate', 'Average Critical Violation Rate',
                              'Inspection day Count', 'Inspection Days With Violation',
                              'Public Health Hazard Violation Count', 'Critical Violation Count', 'General Violation Count',  'Total Violation Count',
                              'Safety score', 'Inspection Data Per Inspection']

In [201]:
final_data_for_app

,Day Care ID,Center Name,ID_Name,Address,Phone,URL,Borough,Zipcode,Latitude,Longitude,Coordinates,_coordinate_Point,Date Permitted,Status,Permit Expiration,Program Type,Facility Type,Child Care Type,Age Range,Maximum Capacity,Max Capacity Range,Total Educational Workers,Average Total Educational Workers,Total Educational Workers Range,Violation Rate Percent,Average Violation Rate Percent,Public Health Hazard Violation Rate,Average Public Health Hazard Violation Rate,Critical Violation Rate,Average Critical Violation Rate,Inspection day Count,Inspection Days With Violation,Public Health Hazard Violation Count,Critical Violation Count,General Violation Count,Total Violation Count,Safety score,Inspection Data Per Inspection
0,DC1000,BILLY MARTIN CHILD DEVELOPMENT CENTER,DC1000: BILLY MARTIN CHILD DEVELOPMENT CENTER,"333 Classon Ave, Brooklyn, NY 11205, USA",718-857-5630,None,BROOKLYN,11205,40.689851,-73.959846,"(40.6898512, -73.95984589999999)",POINT (40.6898512 -73.95984589999999),2004-03-30,Permitted,2024-04-06,PRESCHOOL,GDC,Child Care - Pre School,None,70,50-100,7,11.1888,5-8,50.0000,20.7182,0.0,6.2024,50.0000,17.8904,8,3.0,0.0,5.0,2.0,7.0,9.997075,"[{'Inspection Date': '2022-03-08', 'Violation ..."
1,DC1021,NUESTROS NINOS DAY CAR 1,DC1021: NUESTROS NINOS DAY CAR 1,"384 S 4th St, Brooklyn, NY 11211, USA",718-963-1555,None,BROOKLYN,11211,40.707965,-73.952565,"(40.7079651, -73.9525654)",POINT (40.7079651 -73.9525654),2003-11-24,Permitted,2023-08-07,PRESCHOOL,GDC,Child Care - Pre School,2 YEARS - 5 YEARS,162,>100,33,11.1888,>15,0.0000,20.7182,0.0,6.2024,0.0000,17.8904,19,12.0,1.0,11.0,10.0,22.0,9.985842,"[{'Inspection Date': '2022-07-19', 'Violation ..."
2,DC10244,HERBERT G. BIRCH SERVICES,DC10244: HERBERT G. BIRCH SERVICES,"10-24 49th Ave, Long Island City, NY 11101, USA",718-786-1104,None,QUEENS,11101,40.743021,-73.953118,"(40.74302120000001, -73.95311819999999)",POINT (40.74302120000001 -73.95311819999999),2003-12-01,Permitted,2023-12-04,PRESCHOOL,GDC,Child Care - Pre School,2 YEARS - 5 YEARS,144,>100,26,11.1888,>15,40.0000,20.7182,0.0,6.2024,40.0000,17.8904,8,2.0,0.0,2.0,0.0,2.0,9.999705,"[{'Inspection Date': '2022-06-16', 'Violation ..."
3,DC1025,"CNC CARE CENTER, INC.","DC1025: CNC CARE CENTER, INC.","647 Rogers Ave, Brooklyn, NY 11226, USA",718-284-0172,None,BROOKLYN,11226,40.655723,-73.952801,"(40.6557228, -73.95280129999999)",POINT (40.6557228 -73.95280129999999),2005-09-14,Permitted,2023-09-14,PRESCHOOL,GDC,Child Care - Pre School,2 YEARS - 5 YEARS,19,11-30,3,11.1888,1-5,20.0000,20.7182,0.0,6.2024,20.0000,17.8904,5,2.0,0.0,2.0,2.0,4.0,9.998970,"[{'Inspection Date': '2022-02-09', 'Violation ..."
4,DC1030,"PHIPPS NEIGHBORHOODS, INC.","DC1030: PHIPPS NEIGHBORHOODS, INC.","921 E 180th St, The Bronx, NY 10460, USA",718-364-2496,phippsny.org,BRONX,10460,40.844704,-73.881399,"(40.8447042, -73.88139939999999)",POINT (40.8447042 -73.88139939999999),2002-11-25,Permitted,2023-04-05,PRESCHOOL,GDC,Child Care - Pre School,2 YEARS - 5 YEARS,54,50-100,11,11.1888,8-15,0.0000,20.7182,0.0,6.2024,0.0000,17.8904,11,6.0,0.0,2.0,10.0,12.0,9.995257,"[{'Inspection Date': '2022-06-16', 'Violation ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,DC955,"TUTOR TIME LEARNING CENTERS, LLC","DC955: TUTOR TIME LEARNING CENTERS, LLC","7817 3rd Ave, Brooklyn, NY 11209, USA",718-680-6566,www.tutortime.com,BROOKLYN,11209,40.628756,-74.028576,"(40.6287556, -74.02857589999999)",POINT (40.6287556 -74.02857589999999),2003-06-20,Permitted,2023-08-10,INFANT TODDLER,GDC,Child Care - Infants/Toddlers,0 YEARS - 2 YEARS,36,30-50,20,7.8268,>15,50.0000,17.6510,25.0,5.5585,50.0000,15.2438,6,2.0,2.0,3.0,1.0,6.0,9.950950,"[{'Inspection Date': '2022-02-24', 'Violation ..."
3079,DC956,BAY RIDGE AVE. CHILD CARE CENTER (I/T),DC956: BAY RIDGE AVE. CHILD CARE CENTER (I/T),"1206 Bay Ridge Ave, Brooklyn, NY 11219, USA",718-680-3544,None,BROOKLYN,11219,40.624768,-7

In [202]:
import dill

with open('./data/final_data_for_app.pkl', 'wb') as fd:
    dill.dump(final_data_for_app, fd)
    

In [2]:
import dill   
with open('./data/final_data_for_app.pkl', 'rb') as fb:
    final_data_for_app = dill.load(fb)

In [3]:
final_data_for_app.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3083 entries, 0 to 3082
Data columns (total 38 columns):
 #   Column                                       Non-Null Count  Dtype   
---  ------                                       --------------  -----   
 0   Day Care ID                                  3083 non-null   object  
 1   Center Name                                  3083 non-null   object  
 2   ID_Name                                      3083 non-null   object  
 3   Address                                      3083 non-null   object  
 4   Phone                                        3079 non-null   object  
 5   URL                                          1595 non-null   object  
 6   Borough                                      3083 non-null   object  
 7   Zipcode                                      3080 non-null   object  
 8   Latitude                                     3083 non-null   float64 
 9   Longitude                                    3083 non-null   fl

In [205]:
# 'Day Care ID', 'Center Name', 'Address', 'Phone', 'URL',
#        'Borough', 'Zipcode', 'Date Permitted', 'Status', 'Permit Expiration',
#        'Program Type', 'Facility Type', 'Child Care Type', 'Age Range',
#        'Maximum Capacity',  'Total Educational Workers',
#        'Average Total Educational Workers', 
#        'Violation Rate Percent', 'Average Violation Rate Percent',
#        'Public Health Hazard Violation Rate',
#        'Average Public Health Hazard Violation Rate',
#        'Critical Violation Rate', 'Average Critical Violation Rate',
#        'Inspection day Count', 'Inspection Days With Violation',
#        'Public Health Hazard Violation Count', 'Critical Violation Count',
#        'General Violation Count', 'Total Violation Count',
#        'Inspection Data Per Inspection'

In [9]:
import pandas as pd 

In [6]:
final_data_for_app.loc[0][['Total Educational Workers','Average Total Educational Workers']]

Total Educational Workers                  7
Average Total Educational Workers    11.1888
Name: 0, dtype: object

In [ ]:
final_data_for_app[data_f['ID_Name'].isin([s_choice])][['Day Care ID', 'Center Name', 'Address', 'Phone', 'URL',
        'Borough', 'Zipcode', 'Date Permitted', 'Status', 'Permit Expiration',
        'Program Type', 'Facility Type', 'Child Care Type', 'Age Range',
        'Maximum Capacity',  'Total Educational Workers',
        'Average Total Educational Workers',
        'Violation Rate Percent', 'Average Violation Rate Percent',
        'Public Health Hazard Violation Rate',
        'Average Public Health Hazard Violation Rate',
        'Critical Violation Rate', 'Average Critical Violation Rate',
        'Inspection day Count', 'Inspection Days With Violation',
        'Public Health Hazard Violation Count', 'Critical Violation Count',
        'General Violation Count', 'Total Violation Count',
        'Inspection Data Per Inspection']]